# Alignment-based Measure

In this notebook, we adapt the alignment-based measure that was introduced by van der Aalst et al. in 2012  in the paper: "Replaying history on process models for conformance checking and performance analysis" (doi: https://doi.org/10.1002/widm.1045). <br>
The measure is defined as follows:
<br>
For event log $E$, set of unique activities $E'$, process model $M$, $|\text{sim}(e)|$ as the number of times a state $s$ was visited by the log and $|\text{diff}(e)|$ as the number of unique activities when leaving state $s$, generalization is defined as:

$$Generalization(E,M) = 1-\frac{1}{|E|}\sum_{e\,\text{in}\,E'}|e\,\text{in}\,E|\,*\,{pnew}(|\text{diff}(e)|,|\text{sim}(e)|),\; \text{where} $$
$$pnew(w,n)\begin{cases}
    \frac{w(w+1)}{n(n-1)} & \text{if } n\geq w+2, \\
    1 & \text{otherwise}.
\end{cases}$$ 

If generalization converges towards $0$, it is highly likely that a new event occurs in state $s$, and if generalization converges towards $1$, it is highly unlikely.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from ocpa.objects.log.importer.ocel import factory as ocel_import_factory
from ocpa.algo.discovery.ocpn import algorithm as ocpn_discovery_factory
import numpy as np
from src.models.baseline_measure import baseline_measure
from src.utils import get_happy_path_log, create_flower_model

# O2C Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [3]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [32]:
ots = ["order","item","delivery"]
flower_ocpn = create_flower_model(filename,ots)

In [103]:
#list for values in sum of formula
pnew = []
# We only calculate the values for "non-silent" transitions
transitions = [x for x in flower_ocpn.transitions if not x.silent]
# dictionary to store each activity as key and a list of its prior states/places as value
targets = {}

In [104]:
for arc in flower_ocpn.arcs:
    # for each arc, check if our target is a valid (non-silent) transition
    if arc.target in transitions:
        # load all the prior places of a valid transition into a dictionary, where the key is the transition and the value
        # a list of all directly prior places
        if arc.target.name in targets:
            targets[arc.target.name].append(arc.source.name)
        else:
            targets[arc.target.name] = [arc.source.name]

In [105]:
# get a list of all activities that have been performed in the log
log = ocel.log.log.event_activity
# for each valid transition/event -> for computing reasons(efficiency), we work with a small difference to above
for event in transitions:
    # create an empty list where we can store all enabled transitions in the specific prior place
    enabled= []
    # print(event) #used for debugging
    # get the list of all events that are simultaneously in the current state
    for key in targets:
        # we check if the value is the same or if the value of another key is a subset, because then it is also enabled
        if (targets[event.name] == targets[key]) or (set(targets[key]).issubset(set(targets[event.name]))):
            enabled.append(key)
    # number of activities that can be triggered
    w = len(enabled)
    # number of times this state is visited in the log
    n = len(log[log.isin(enabled)])
    # frequency of event that is currently watched
    freq = len(log[log == event.name])
    #print(w) #used for debugging
    #print(n) #used for debugging
    #print(freq) #used for debugging
    # derive the value for the sum in the formula given
    if n >= w+2 :
        pnew.append(freq*(w*(w+1))/(n*(n-1)))
    else:
        pnew.append(freq*1)
#derive the final generalization value
generalization = 1 - np.sum(pnew)/len(log)

In [108]:
generalization

0.9999739979667729

In [110]:
def alignment_measure(ocel,ocpn):
    #list for values in sum of formula
    pnew = []
    # get a list of all activities that have been performed in the log
    log = ocel.log.log.event_activity
    # We only calculate the values for "non-silent" transitions
    transitions = [x for x in flower_ocpn.transitions if not x.silent]
    # dictionary to store each activity as key and a list of its prior states/places as value
    targets = {}
    for arc in flower_ocpn.arcs:
        # for each arc, check if our target is a valid (non-silent) transition
        if arc.target in transitions:
            # load all the prior places of a valid transition into a dictionary, where the key is the transition and the value
            # a list of all directly prior places
            if arc.target.name in targets:
                targets[arc.target.name].append(arc.source.name)
            else:
                targets[arc.target.name] = [arc.source.name]
    # for each valid transition/event -> for computing reasons(efficiency), we work with a small difference to above
    for event in transitions:
        # create an empty list where we can store all enabled transitions in the specific prior place
        enabled= []
        # print(event) #used for debugging
        # get the list of all events that are simultaneously in the current state
        for key in targets:
            # we check if the value is the same or if the value of another key is a subset, because then it is also enabled
            if (targets[event.name] == targets[key]) or (set(targets[key]).issubset(set(targets[event.name]))):
                enabled.append(key)
        # number of activities that can be triggered
        w = len(enabled)
        # number of times this state is visited in the log
        n = len(log[log.isin(enabled)])
        # frequency of event that is currently watched
        freq = len(log[log == event.name])
        #print(w) #used for debugging
        #print(n) #used for debugging
        #print(freq) #used for debugging
        # derive the value for the sum in the formula given
        if n >= w+2 :
            pnew.append(freq*(w*(w+1))/(n*(n-1)))
        else:
            pnew.append(freq*1)
    #derive the final generalization value
    return (1 - np.sum(pnew)/len(log))

In [111]:
generalization = alignment_measure (ocel,flower_ocpn)

In [112]:
generalization

0.9999739979667729